In [1]:
using DifferentialEquations, Plots, ReferenceFrameRotations, FiniteDiff, ForwardDiff, BenchmarkTools

In [2]:
# This code is for simulating a chain of N points.
# This is to emulate a receptor where the points represent
# the hinges.
# Each hinge can have a defined potential in the local
# spherical coordinates, which is converted to global 
# Cartesian via rotation quaternions and directional cosine
# matrices.
# The simulation is carried out in the global Cartesian
# coordinate system.

# Define some basic coordinate identities
function CartesianToSpherical(X::Vector)
    r = sqrt(X[1]^2 + X[2]^2 + X[3]^2)
    θ = angle(X[1] + im*X[2])
    ϕ = acos(X[3]/r)
    return [r,θ,ϕ]
end

function SphericalToCartesian(Φ::Vector)
    x = Φ[1]*cos(Φ[2])*sin(Φ[3])
    y = Φ[1]*sin(Φ[2])*sin(Φ[3])
    z = Φ[1]*cos(Φ[3])
    return [x,y,z]
end

function StCθ(Φ::Vector)
    x = - Φ[1]*sin(Φ[2])*sin(Φ[3])
    y = Φ[1]*cos(Φ[2])*sin(Φ[3])
    z = 0
    return [x,y,z]
end

function StCϕ(Φ::Vector)
    x = Φ[1]*cos(Φ[2])*cos(Φ[3])
    y = Φ[1]*sin(Φ[2])*cos(Φ[3])
    z = - Φ[1]*sin(Φ[3])
    return [x,y,z]
end

function StCr(Φ::Vector)
    x = cos(Φ[2])*sin(Φ[3])
    y = sin(Φ[2])*sin(Φ[3])
    z = cos(Φ[3])
    return [x,y,z]
end

# Find the directional cosine matrix (DCM) for a given set of angles and
# a set of initial basis vectors.
function FindDCM(θ,ϕ,K::Matrix{Float64})
    I3 = 1.0*Matrix(I,3,3)
    qϕ = Quaternion(cos(ϕ/2), sin(ϕ/2)*K[2,:]) # Rotate by ϕₙ' around ŷ(n-1)
    qθ = Quaternion(cos(θ/2), sin(θ/2)*K[3,:]) # Rotate by θₙ' around ẑ(n-1)
    q = qθ*qϕ

    for n in 1:3
        K[n,:] = vect(q*K[n,:]/q)
    end
    
    return [I3[n,:]' * K[m,:] for n in 1:3, m in 1:3], K
end

# Algorithm to convert Cartesian coordinates of hinge points and find 
# their local spherical coordinates.
function FindRecLocals(X::Matrix)
    Locals = reshape(CartesianToSpherical(X[1,:]),1,3)
    # D, K, q = FindDCM(Locals[1,2], Locals[1,3], 1.0*Matrix(I,3,3))
    D = 1.0*Matrix(I,3,3)
    K = 1.0*Matrix(I,3,3)
    Q = reshape([],2,0)

    for n in 2:size(X)[1]
        Locals = [Locals; CartesianToSpherical(D' * (X[n,:] - X[n-1,:]))']
        D, K = FindDCM(Locals[n,2], Locals[n,3], copy(K))
    end
    return Locals
end

function FindLocals(X::Matrix,N::Integer)
    NR = trunc(Int, (size(X,1)-1)/N)
    ξ = [[X[1,:]'; X[1+N*(nr-1)+1:1+N*nr,:]] for nr in 1:NR]
    Locals = Array{Float64}(undef,0,3)

    for x in ξ
        Locals = vcat(Locals,FindRecLocals(x))
    end

    return vcat(Locals[1,:]', Locals[(1:end).%(N+1) .!= 1, :])
end

function FindRecCartesians(Φ::Matrix)
    D, K = 1.0*Matrix(I,3,3), 1.0*Matrix(I,3,3) # FindDCM(Φ[1,2], Φ[1,3], 1.0*Matrix(I,3,3))
    DCMs = [copy(D)]
    Carts = reshape(SphericalToCartesian(Φ[1,:]),1,3)

    for n in 2:size(Φ,1)
        D, K = FindDCM(Φ[n,2], Φ[n,3], K)
        Carts = [Carts; reshape(Carts[n-1,:] + DCMs[n-1]*SphericalToCartesian(Φ[n,:]),1,3)]
        push!(DCMs, copy(D))
    end

    return Carts
end

function FindCartesians(Φ::Matrix,N::Integer)
    NR = trunc(Int, (size(Φ,1)-1)/N)
    P = [[Φ[1,:]';Φ[1+N*(nr-1)+1:1+N*nr,:]] for nr in 1:NR]
    Carts = Array{Float64}(undef,0,3)

    for ϕ in P
        Carts = vcat(Carts,FindRecCartesians(ϕ))
    end

    return vcat(copy(Carts[1,:]'), Carts[(1:end).%(N+1) .!= 1, :])
end

# Jacobian of a single receptor
function JacobianMinor(U,N)
    Temp = FiniteDiff.finite_difference_jacobian(FindRecCartesians, U)
    J = zeros(size(Temp))
    for n in 1:N
        J[3*n-2,:] = Temp[n,:]
        J[3*n-1,:] = Temp[N+n,:]
        J[3*n,:] = Temp[2*N+n,:]
    end
    Temp = copy(J)
    for n in 1:N
        J[:,3*n-2] = Temp[:,n]
        J[:,3*n-1] = Temp[:,N+n]
        J[:,3*n] = Temp[:,2*N+n]
    end
    return J
end

function ModularJacobian(U,N,Nr)
    # Jacobian = zeros(3*N*Nr+3, 3*N*Nr+3)
    JList = []
    for nr in 1:Nr
        Jm = JacobianMinor(vcat(copy(U[1,:]'),U[1+N*(nr-1)+1:1+N*nr,:]),N+1)
        push!(JList, Jm)
        # Jacobian[3+3*N*(nr-1)+1:3+3*N*nr,1:3] = Jm[4:end,1:3]
        # Jacobian[3+3*N*(nr-1)+1:3+3*N*nr,3+3*N*(nr-1)+1:3+3*N*nr] = Jm[4:end,4:end]
    end
    # Jacobian[1:3,1:3] = [StCr(U[1,:]) StCθ(U[1,:]) StCϕ(U[1,:])]

    return JList # , Jacobian
end

# Calculate the Jacobian of dΦ = J*dX
# function Jacobian(U::Matrix{Float64},N::Integer,Nr::Integer)
#     Temp = FiniteDiff.finite_difference_jacobian(x -> FindCartesians(x,N), U)
#     J = zeros(size(Temp))
#     J[1,:] = copy(Temp[1,:])
#     J[2,:] = copy(Temp[1+N*Nr+1,:])
#     J[3,:] = copy(Temp[2*(1+N*Nr)+1,:])
#     for nr in 1:Nr
#         for n in 1:N
#             J[3+3*N*(nr-1)+3*n-2,:] = Temp[N*(nr-1)+1+n,:]        # x of receptor nr, particle n
#             J[3+3*N*(nr-1)+3*n-1,:] = Temp[(1+N*Nr)+N*(nr-1)+1+n,:]  # y of receptor nr, particle n
#             J[3+3*N*(nr-1)+3*n,:] = Temp[2*(1+N*Nr)+N*(nr-1)+1+n,:]         # z of receptor nr, particle n
#         end
#     end
#     Temp = copy(J)
#     J[:,1] = copy(Temp[:,1])
#     J[:,2] = copy(Temp[:,1+N*Nr+1])
#     J[:,3] = copy(Temp[:,2*(1+N*Nr)+1])
#     for nr in 1:Nr
#         for n in 1:N
#             J[:,3+3*N*(nr-1)+3*n-2] = Temp[:,N*(nr-1)+1+n]        # x of receptor nr, particle n
#             J[:,3+3*N*(nr-1)+3*n-1] = Temp[:,(1+N*Nr)+N*(nr-1)+1+n]  # y of receptor nr, particle n
#             J[:,3+3*N*(nr-1)+3*n] = Temp[:,2*(1+N*Nr)+N*(nr-1)+1+n]         # z of receptor nr, particle n
#         end
#     end
#     return J
# end

ModularJacobian (generic function with 1 method)

In [3]:
k = 10
κ = 1000
D = 0.1
R = 1
N = 0     # Number of joints on receptor
NRec = 0  # Number of receptors

t0 = 0.0
tf = 10
dt = 1//1000
u0 = rand(3*(1+N*NRec)) # [0,0,eps(),0,0,1,0,0,2,0,0,-1,0,0,-2]

function ReceptorV(u,N)
    # Insert localised forces here
    # Function of (r,θ,ϕ)ₙ' for all n ∈ [1,N]
    du = zeros(length(u))

    # The second 3 represent the base of the receptor
    du[1] = -κ*(u[1] - R) - κ*(u[4] - R)
    du[2] = 0
    du[3] = 0

    for n in 4:3:3*N-3
        du[n] = -κ*(u[n]-R) - κ*(u[n+3]-R)
        du[n+1] = 0
        du[n+2] = -k*u[n+2]
    end

    du[end-2] = -κ*(u[end-2]-R)
    du[end-1] = 0
    du[end] = -k*u[end]

    return du
end

function VirusV(u,NR)
    dϕ = zeros(length(u)) # *NR - (NR-1))

    # These first 3 represent the centre of the virus
    dϕ[1] = 0 # -κ*(u[4] - R)  # r₀'
    dϕ[2] = 0              # θ₀'
    dϕ[3] = 0              # ϕ₀'

    # The receptors
    for nr in 1:NR
        RBlock = 3*N*(nr-1)
        dϕ[3+RBlock+1:3+RBlock+3*N] = ReceptorV(u[3+RBlock+1:3+RBlock+3*N],N)
    end
    return dϕ
end

function det(du,u,p,t)
    Φ = FindLocals(copy(reshape(u,3,1+N*NRec)'),N)
    Φ_vect = reshape(copy(Φ'),length(Φ))
    V = VirusV(Φ_vect, NRec)   # Define potential' (force) in terms of local spherical
    JList = ModularJacobian(Φ,N,NRec)
    for nr in 1:NRec
        V[3+3*N*(nr-1)+1:3+3*N*nr] = JList[nr][4:end,1:end]*[V[1:3]; V[3+3*N*(nr-1)+1:3+3*N*nr]]
    end

    if mod(trunc(t/dt),trunc(tf/dt/5)) == 0
        # display(trunc(1000*t))
        println(String("$(trunc(Int,t/tf*100))%"))
    end

    du[1:end] = V[1:end]
end

function stoch(du,u,p,t)
    du[1:3] .= 0
    # du[4:6] .= sqrt(6*D)
    du[4:end] .= sqrt(2*D)
end

TimeRange = range(t0,tf,step = dt)[1:end-1]
prob = SDEProblem(det,stoch,u0,(t0,tf))
display("Problem defined")
sol = solve(prob,EM(),dt=dt)
display("Problem solved")

SN = []

for i in 1:1+N*NRec
    push!(SN, [sol.u[n][m] for n in 1:length(sol), m in 1+3*(i-1):3+3*(i-1)])
end

plotly()
P = plot(SN[1][:,1], SN[1][:,2], SN[1][:,3],
    legend = :none,
    xlabel="x",
    ylabel="y",
    zlabel="z")

for n in 2:1+N*NRec
    plot!(SN[n][:,1], SN[n][:,2], SN[n][:,3])
end
display(P)

"Problem defined"

LoadError: InexactError: trunc(Int64, NaN)

In [26]:
let 
    N = 10
    Nr = 50
    u = rand(1+N*Nr,3)
    Φ = FindLocals(u,N)

    # JM = round.(ModularJacobian(Φ,N,Nr), digits=2)
    # JO = round.(Jacobian(Φ,N,Nr), digits=2)
    # show(IOContext(stdout, :limit=>false), MIME"text/plain"(), norm(JM-JO))

    display(@benchmark $ModularJacobian(rand(1+10*10,3),10,10))
    @benchmark $Jacobian(rand(1+10*10,3),10,10)
end

BenchmarkTools.Trial: 580 samples with 1 evaluation.
 Range (min … max):  6.138 ms … 21.809 ms  ┊ GC (min … max):  0.00% … 44.63%
 Time  (median):     7.720 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   8.618 ms ±  2.818 ms  ┊ GC (mean ± σ):  11.92% ± 16.72%

    ▁ ▃▂▂▃▄█▂▂                                                
  ▅▆██████████▇▄▃▂▂▃▁▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂▂▂▃▃▃▄▄▄▄▃▃▃▂▁▁▂ ▃
  6.14 ms        Histogram: frequency by time        16.6 ms <

 Memory estimate: 13.72 MiB, allocs estimate: 136156.

BenchmarkTools.Trial: 40 samples with 1 evaluation.
 Range (min … max):  116.114 ms … 138.768 ms  ┊ GC (min … max): 14.94% … 18.57%
 Time  (median):     125.517 ms               ┊ GC (median):    14.03%
 Time  (mean ± σ):   125.954 ms ±   5.653 ms  ┊ GC (mean ± σ):  15.29% ±  2.33%

       ▃    ▃ ▃▃    ▃    █   ▃     ▃  ▃                          
  ▇▁▁▁▁█▇▁▇▁█▁██▁▇▁▇█▇▁▁▇█▁▁▁█▇▇▇▇▁█▇▇█▇▇▁▁▇▁▁▁▁▁▁▁▇▁▁▇▇▁▁▁▇▁▁▇ ▁
  116 ms           Histogram: frequency by time          139 ms <

 Memory estimate: 215.58 MiB, allocs estimate: 1191984.

In [89]:
let
    function ModularJacobian(U,N,Nr)
        Jacobian = zeros(3*N*Nr+3, 3*N*Nr+3)
        for nr in 1:Nr
            Jm = JacobianMinor(vcat(copy(U[1,:]'),U[1+N*(nr-1)+1:1+N*nr,:]),N+1)
            Jacobian[3+3*N*(nr-1)+1:3+3*N*nr,1:3] = Jm[4:end,1:3]
            Jacobian[3+3*N*(nr-1)+1:3+3*N*nr,3+3*N*(nr-1)+1:3+3*N*nr] = Jm[4:end,4:end]
        end
        Jacobian[1:3,1:3] = [StCr(U[1,:]) StCθ(U[1,:]) StCϕ(U[1,:])]
    
        return Jacobian
    end

    function det(du,u,p,t)
        Φ = FindLocals(copy(reshape(u,3,1+N*NRec)'),N)
        Φ_vect = reshape(copy(Φ'),length(Φ))
        V = VirusV(Φ_vect, NRec)   # Define potential' (force) in terms of local spherical
        V = ModularJacobian(Φ,N,NRec) * V

        # if mod(trunc(t/dt),trunc(tf/dt/5)) == 0
        #     # display(trunc(1000*t))
        #     println(String("$(trunc(Int,t/tf*100))%"))
        # end

        du[1:end] = V[1:end]
    end

    @benchmark solve($prob,EM(),dt=$dt) seconds=60
end

BenchmarkTools.Trial: 47 samples with 1 evaluation.
 Range (min … max):  1.109 s …    1.898 s  ┊ GC (min … max): 10.91% … 10.40%
 Time  (median):     1.248 s               ┊ GC (median):    11.37%
 Time  (mean ± σ):   1.295 s ± 137.904 ms  ┊ GC (mean ± σ):  11.34% ±  0.50%

       ▅ ▂▂█▅   ▂                                             
  ▅▁▅▅▅████████▅██▁█▅▁▅▁▁█▅▁▁▁█▁▅▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  1.11 s         Histogram: frequency by time          1.9 s <

 Memory estimate: 1.29 GiB, allocs estimate: 15317059.

In [90]:
let
    function ModularJacobian(U,N,Nr)
        JList = []
        for nr in 1:Nr
            Jm = JacobianMinor(vcat(copy(U[1,:]'),U[1+N*(nr-1)+1:1+N*nr,:]),N+1)
            push!(JList, Jm)
        end
        Jacobian[1:3,1:3] = [StCr(U[1,:]) StCθ(U[1,:]) StCϕ(U[1,:])]
    
        return JList
    end

    function det(du,u,p,t)
        Φ = FindLocals(copy(reshape(u,3,1+N*NRec)'),N)
        Φ_vect = reshape(copy(Φ'),length(Φ))
        V = VirusV(Φ_vect, NRec)   # Define potential' (force) in terms of local spherical
        JList = ModularJacobian(Φ,N,NRec)
        for nr in 1:NRec
            V[3+3*N*(nr-1)+1:3+3*N*nr] = JList[nr][4:end,1:end]*[V[1:3]; V[3+3*N*(nr-1)+1:3+3*N*nr]]
        end

        # if mod(trunc(t/dt),trunc(tf/dt/5)) == 0
        #     # display(trunc(1000*t))
        #     println(String("$(trunc(Int,t/tf*100))%"))
        # end

        du[1:end] = V[1:end]
    end

    @benchmark solve($prob,EM(),dt=$dt) seconds=60
end

BenchmarkTools.Trial: 52 samples with 1 evaluation.
 Range (min … max):  1.066 s …   1.421 s  ┊ GC (min … max): 11.88% … 12.10%
 Time  (median):     1.139 s              ┊ GC (median):    11.71%
 Time  (mean ± σ):   1.162 s ± 82.758 ms  ┊ GC (mean ± σ):  11.72% ±  0.44%

         █ █▂ █  ▂ █                                         
  ▅▁███▅▁████▁█▅██▅█▁█▅▁▁▁▁▁▅▅▁▁▁▁█▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▅ ▁
  1.07 s         Histogram: frequency by time         1.4 s <

 Memory estimate: 1.29 GiB, allocs estimate: 15317059.